<a href="https://colab.research.google.com/github/akbarriki/Scraping-SIDATA-PTN/blob/main/Scraping_SIDATA_PTN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import requests, json
import numpy as np
import warnings, re
from bs4 import BeautifulSoup
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [2]:
BASE_URL = "https://sidata-ptn.ltmpt.ac.id/ptn_sn.php"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [3]:
resp = requests.get(BASE_URL, headers=HEADERS, verify=False)
resp.status_code

200

In [8]:
results = []
soup = BeautifulSoup(resp.content, 'html.parser')

for tr in soup.select("table > tbody > tr"):
  _, kode_univ, univ_nama_portal, _ = [td.get_text().strip() for td in tr.find_all("td")]
  univ_nama, univ_portal = univ_nama_portal.split("\n")
  univ_portal = re.findall("\(([\w\:\-\_\/\.]+)\s*\)", univ_portal)[0]

  PTN_URL = BASE_URL + f"?ptn={kode_univ}"
  ptn_resp = requests.get(PTN_URL, headers=HEADERS, verify=False)
  if ptn_resp.ok:
    print(univ_nama)
    ptn_soup = BeautifulSoup(ptn_resp.content, 'html.parser')
    for _, ptn_tr in enumerate(tqdm(ptn_soup.select("table > tbody > tr"))):
      _, kode_prodi, nama_prodi, jenjang, daya_tampung_2023, _,_ = [td.get_text().strip() for td in ptn_tr.find_all("td")]

      PRODI_URL = PTN_URL + f"&prodi={kode_prodi}&jenis=0"
      prodi_resp = requests.get(PRODI_URL, headers=HEADERS, verify=False)
      if prodi_resp.ok:
        prodi_soup = BeautifulSoup(prodi_resp.content, 'html.parser')
        minat_daya_tampung = []
        for i, prodi_tr in enumerate(prodi_soup.select("div.panel-info > div.panel-body > table.table-condensed > tbody > tr")):
          minat_daya_tampung.append([td.get_text().strip() for i, td in enumerate(prodi_tr.find_all("td")) if i > 0])

        try:
          minat = {"peminat_"+str(2018+i):int(el) for i, el in enumerate(minat_daya_tampung[0])}
          daya_tampung = {"daya_tampung_"+str(2018+i):int(el.split("(")[0].strip()) for i, el in enumerate(minat_daya_tampung[1])}
          dct = {
              'kode_univ': kode_univ, 'nama_univ': univ_nama, 'portal_univ': univ_portal,
              'kode_prodi': kode_prodi, 'nama_prodi': nama_prodi, 'jenjang': jenjang, 'daya_tampung_2023': daya_tampung_2023
          }
          dct.update(minat)
          dct.update(daya_tampung)
          results.append(dct)
        except:
          continue

UNIVERSITAS SYIAH KUALA


100%|██████████| 77/77 [00:49<00:00,  1.56it/s]


UNIVERSITAS MALIKUSSALEH


100%|██████████| 35/35 [00:23<00:00,  1.52it/s]


UNIVERSITAS TEUKU UMAR


...

100%|██████████| 27/27 [00:14<00:00,  1.86it/s]


ISBI TANAH PAPUA


100%|██████████| 5/5 [00:02<00:00,  1.98it/s]


UNIVERSITAS PAPUA


100%|██████████| 46/46 [00:28<00:00,  1.62it/s]


In [11]:
df = pd.DataFrame(results)
df.head()

,kode_univ,nama_univ,portal_univ,kode_prodi,nama_prodi,jenjang,daya_tampung_2023,peminat_2018,peminat_2019,peminat_2020,peminat_2021,peminat_2022,daya_tampung_2018,daya_tampung_2019,daya_tampung_2020,daya_tampung_2021,daya_tampung_2022
0,111,UNIVERSITAS SYIAH KUALA,https://pmb.usk.ac.id/,111001,PENDIDIKAN DOKTER HEWAN,Sarjana,50,633,338,413,391,389,66,66,55,55,55
1,111,UNIVERSITAS SYIAH KUALA,https://pmb.usk.ac.id/,111002,TEKNIK SIPIL,Sarjana,54,1097,554,571,498,435,45,48,45,45,45
2,111,UNIVERSITAS SYIAH KUALA,https://pmb.usk.ac.id/,111003,TEKNIK MESIN,Sarjana,30,336,199,162,229,172,30,36,35,35,35
3,111,UNIVERSITAS SYIAH KUALA,https://pmb.usk.ac.id/,111004,TEKNIK KIMIA,Sarjana,35,415,201,218,180,174,30,36,35,35,35
4,111,UNIVERSITAS SYIAH KUALA,https://pmb.usk.ac.id/,111005,ARSITEKTUR,Sarjana,38,559,314,314,326,303,30,36,35,35,35


In [14]:
df_univ = df.iloc[:, :3].copy()
df_univ.drop_duplicates(inplace=True)
df_univ.head()

,kode_univ,nama_univ,portal_univ
0,111,UNIVERSITAS SYIAH KUALA,https://pmb.usk.ac.id/
63,112,UNIVERSITAS MALIKUSSALEH,http://pmb.unimal.ac.id/
93,113,UNIVERSITAS TEUKU UMAR,https://pmb.utu.ac.id/
111,114,UNIVERSITAS SAMUDRA,https://pmb.unsam.ac.id/
136,115,ISBI ACEH,http://pmb.isbiaceh.ac.id/


In [15]:
len(df_univ)

84

In [22]:
df_prodi = df[[col for col in df.columns if col not in ['nama_univ','portal_univ']]].copy()
df_prodi.drop_duplicates(inplace=True)
df_prodi.head()

,kode_univ,kode_prodi,nama_prodi,jenjang,daya_tampung_2023,peminat_2018,peminat_2019,peminat_2020,peminat_2021,peminat_2022,daya_tampung_2018,daya_tampung_2019,daya_tampung_2020,daya_tampung_2021,daya_tampung_2022
0,111,111001,PENDIDIKAN DOKTER HEWAN,Sarjana,50,633,338,413,391,389,66,66,55,55,55
1,111,111002,TEKNIK SIPIL,Sarjana,54,1097,554,571,498,435,45,48,45,45,45
2,111,111003,TEKNIK MESIN,Sarjana,30,336,199,162,229,172,30,36,35,35,35
3,111,111004,TEKNIK KIMIA,Sarjana,35,415,201,218,180,174,30,36,35,35,35
4,111,111005,ARSITEKTUR,Sarjana,38,559,314,314,326,303,30,36,35,35,35


In [23]:
df_univ.to_csv("daftar_universitas.csv", index=False)
df_prodi.to_csv("daftar_prodi.csv", index=False)